In [185]:
# imports 

import os
import numpy as np
from bs4 import BeautifulSoup
from lxml import etree
import string
import pickle
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maggiebarclay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [118]:
# get all of the info from html files
# two lists, one of the names one with the descriptions

characterList = []
descriptionsList = []

for filename in os.listdir("project_crawler/project_crawler/htmlFiles"):
    try:
        #print("reading file: " + str(filename))
        with open("project_crawler/project_crawler/htmlFiles/" + str(filename), 'r') as f:
            soup = BeautifulSoup(f, 'html.parser')
            character = soup.title.getText().split()[0]
            characterList.append(character)

            # appending separate, not sure if getting all description parts yet or just the first
            descList = []
            descList.append(soup.find_all('p')[1].getText().replace('\n',' '))
            descList.append(soup.find_all('p')[2].getText().replace('\n',' '))
            descList.append(soup.find_all('p')[3].getText().replace('\n',' '))

            description = "".join(descList)
            descriptionsList.append(description)

    except:
        print(filename)
        print("This is being skipped,likely because of an unrecognized letter/character")
        pass


#print(characterList)
print(len(characterList))
#print(descriptionsList)
print(len(descriptionsList))

.DS_Store
This is being skipped,likely because of an unrecognized letter/character
463
463


In [119]:
descriptionsList[0]

'Bam is a jock deer villager in the Animal Crossing series. He first appeared in Animal Crossing: New Leaf, and he was one of the first villagers announced for the release of the game. He has appeared in all subsequent games. As of Animal Crossing: New Horizons, he is the only jock deer in the series. His name may be derived from Bambi, a male deer who is the main character of the novel Bambi, a Life in the Woods and its film adaptation. In New Horizons, Bam has the play hobby and can run around with his arms extended at any time, rather than only outside Resident Services during a scripted activity with a group of villagers. Bam is a blue deer. He has lighter blue muzzle and rings around his eyes, orange irises with brown pupils, dark gray freckles, and brown antlers and brown hooves. The insides of his ears are yellow. '

In [120]:
def tokenizerMethod(docString: str):
  noPunctString = (docString.lower()).translate(str.maketrans('', '', string.punctuation))
  tokenizedDoc = word_tokenize(noPunctString)
  return tokenizedDoc

tokenizedDocList = []

for item in descriptionsList:
  tokenizedDocList.append(tokenizerMethod(item))

In [121]:
print(tokenizedDocList[0])

['bam', 'is', 'a', 'jock', 'deer', 'villager', 'in', 'the', 'animal', 'crossing', 'series', 'he', 'first', 'appeared', 'in', 'animal', 'crossing', 'new', 'leaf', 'and', 'he', 'was', 'one', 'of', 'the', 'first', 'villagers', 'announced', 'for', 'the', 'release', 'of', 'the', 'game', 'he', 'has', 'appeared', 'in', 'all', 'subsequent', 'games', 'as', 'of', 'animal', 'crossing', 'new', 'horizons', 'he', 'is', 'the', 'only', 'jock', 'deer', 'in', 'the', 'series', 'his', 'name', 'may', 'be', 'derived', 'from', 'bambi', 'a', 'male', 'deer', 'who', 'is', 'the', 'main', 'character', 'of', 'the', 'novel', 'bambi', 'a', 'life', 'in', 'the', 'woods', 'and', 'its', 'film', 'adaptation', 'in', 'new', 'horizons', 'bam', 'has', 'the', 'play', 'hobby', 'and', 'can', 'run', 'around', 'with', 'his', 'arms', 'extended', 'at', 'any', 'time', 'rather', 'than', 'only', 'outside', 'resident', 'services', 'during', 'a', 'scripted', 'activity', 'with', 'a', 'group', 'of', 'villagers', 'bam', 'is', 'a', 'blue', 

In [122]:
# TFIDF

def calculateDF(tokenizedDocList: list):
  invInd = {}
  index = -1
  for doc in tokenizedDocList:
    index += 1
    for word in list(set(doc)):
      word = (word.lower()).translate(str.maketrans('', '', string.punctuation))
      word = "$" + word + "$"
      if word in invInd:
        currentDF = invInd[word]
        currentDF += 1
        invInd[word] = currentDF
      else: 
        invInd[word] = 1
  return invInd

In [171]:
def makeInvInd(tokenizedDocList: list):
  invInd = {}
  index = -1
  #N is the number of docs 
  N = len(tokenizedDocList)
  dfIndx = calculateDF(tokenizedDocList)

  for doc in tokenizedDocList:
    index += 1
    for word in list(set(doc)):
      formatWord = '$' + word + '$'
      word = (word.lower()).translate(str.maketrans('', '', string.punctuation))
      #tf is how many times term appears in the document / len of doc <- going off of book so not dividing by length
      tf = doc.count(word)
      df = dfIndx[formatWord]
      idf = np.log(N/(df))
      tfidf = (tf*idf)

      if formatWord in invInd:    
        currList = invInd[formatWord]
        currList.append(((str(index),  str(characterList[index]), tfidf)))
        invInd[formatWord] = currList
        
      else:   
        invInd[formatWord] = [(str(index), str(characterList[index]), tfidf)]

  return invInd

In [172]:
invIndex = makeInvInd(tokenizedDocList)  
#print(invIndex["dog"])

# the method below is a user friendly way to lookup, just put in a word as usual and it will do the $ addition for you
def lookupInDocs(term):
  term = "$" + term + "$"
  return invIndex[term]

print(lookupInDocs("dog"))

[('57', 'Lucky', 5.919346447476577), ('75', 'Goldie', 8.879019671214865), ('89', 'Bea', 11.838692894953153), ('97', 'Butch', 14.798366118691442), ('100', 'Portia', 14.798366118691442), ('115', 'Jeremiah', 2.9596732237382883), ('122', 'Holden', 2.9596732237382883), ('151', 'Cobb', 2.9596732237382883), ('187', 'Bones', 8.879019671214865), ('198', 'Marcel', 5.919346447476577), ('203', 'Shep', 11.838692894953153), ('221', 'Vivian', 2.9596732237382883), ('225', 'Mac', 8.879019671214865), ('229', 'Cherry', 8.879019671214865), ('237', 'Cookie', 11.838692894953153), ('293', 'Walker', 5.919346447476577), ('295', 'Daisy', 2.9596732237382883), ('349', 'Maddie', 8.879019671214865), ('351', 'Chai', 2.9596732237382883), ('403', 'Biskit', 8.879019671214865), ('412', 'Frett', 5.919346447476577), ('413', 'Kyle', 5.919346447476577), ('429', 'Marty', 2.9596732237382883), ('452', 'Benjamin', 5.919346447476577)]


In [173]:
def queryVector(docsList, query):
  query = query.split()
  queryIndx = {}
  dfIndx = calculateDF(docsList)

  # idf = log (docs in corpus / docs with term in them)
  N = len(docsList)

  for term in query:
    term = "$" + term + "$"
    df = dfIndx[term]
    idf = np.log(N/(df))

    if term in queryIndx:    
        pass
    else:   
        queryIndx[term] = idf
  return queryIndx

In [174]:
queryIndex = queryVector(tokenizedDocList, "blue dog")
queryIndex

{'$blue$': 0.9119803803730329, '$dog$': 2.9596732237382883}

In [179]:
def cosine(query, index):
    scores = {}

    for query_term, query_weight in query.items():
        for charNum, character, doc_weight in index[query_term]:
          
          if character not in scores: 
            scores[character] = query_weight * doc_weight
            #print(charNum, character, doc_weight)
          else:
            scores[character] += query_weight * doc_weight  
            #print(charNum, character, doc_weight)

    finalScores = {}
    for character in scores.keys():
      finalScores[character] = (float(scores[character]) / (len(tokenizedDocList[int(charNum)])))

    return sorted(finalScores.items(), key=lambda x: x[1], reverse=True)
        
results = cosine(queryIndex, invIndex)
print(results[:10])

[('Butch', 0.4171269329196854), ('Portia', 0.4171269329196854), ('Bea', 0.3337015463357483), ('Shep', 0.3337015463357483), ('Cookie', 0.3337015463357483), ('Goldie', 0.25027615975181117), ('Bones', 0.25027615975181117), ('Mac', 0.25027615975181117), ('Cherry', 0.25027615975181117), ('Maddie', 0.25027615975181117)]


In [186]:
numberOfBatches = 3

def processDocs(docsList, numberOfBatches):
  start = 0
  blockSize = int(len(docsList)/numberOfBatches)

  listOfPairs = []

  for block in range(numberOfBatches):
    for i in range(start, start+blockSize):
      tokenizedDoc = docsList[i]
      for term in tokenizedDoc:
        listOfPairs.append((term, i))

    print("writing block of docs with indecies: " + str(start) + " through " +  str(start+blockSize))
    listfile = open('listPickle','wb')
    pickle.dump(listOfPairs,listfile)
    listfile.close()

    start += blockSize

processDocs(tokenizedDocList, numberOfBatches)

writing block of docs with indecies: 0 through 154
writing block of docs with indecies: 154 through 308
writing block of docs with indecies: 308 through 462


In [191]:

  pickle_in = open('listPickle','rb')
  unpickedLists = pickle.load(pickle_in)

In [189]:
#print(invInd)